# EVCS-OPTIM
Electric Vehicle Charging Station - Optimization Placement Tool via Intersection Modeling

**Data Gathering & Pre-Processing Notebook**

## Initialization

In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import matplotlib.pyplot as plt


import folium
from folium.plugins import HeatMap
import branca
import branca.colormap as cm

import osmnx as ox
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon, mapping, MultiPolygon

## Data Gathering

### Zip Codes

We will need to use SDG&E territory zip codes to filter much of our data by the relevant geographical area

In [2]:
sdge_zips_data = pd.read_excel("data/SDGE Service Territory Zip Code List Q2-2021.xlsx")
sdge_zips = list(sdge_zips_data['ZIP_CODE'])

### Parking Lots

In [3]:
# Query and Plot Parking Lots on a Map

# Define the area of interest
place_name = "San Diego, California, USA"

# Query OSM for parking lots
parking_lots = ox.geometries_from_place(place_name, tags={"amenity": "parking"})

# Filter for public parking lots
public_lots = parking_lots[
    (parking_lots.get("parking") == "public") |  # Explicitly public parking
    (parking_lots.get("access").isin([None, "yes", "permissive"]))  # Exclude private/restricted lots
]

# Convert to a GeoDataFrame
public_lots_gdf = gpd.GeoDataFrame(public_lots, geometry="geometry", crs="EPSG:4326") # Public Lots
parking_lots_gdf = gpd.GeoDataFrame(parking_lots, geometry="geometry", crs="EPSG:4326") # All Lots
parking_cols = ['osmid','geometry','amenity','access','fee','parking','capacity']
public_lots_gdf = public_lots_gdf.reset_index()[parking_cols]
parking_lots_gdf = parking_lots_gdf.reset_index()[parking_cols]

/tmp/ipykernel_504/858120345.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  parking_lots = ox.geometries_from_place(place_name, tags={"amenity": "parking"})


In [4]:
parking_lots_gdf.head(2)

,osmid,geometry,amenity,access,fee,parking,capacity
0,290588762,POINT (-117.19731 32.83273),parking,yes,no,surface,NaN
1,316725319,POINT (-117.15981 32.74716),parking,NaN,NaN,multi-storey,NaN


### Zoning Data

Zoning regulations and land-use can be a useful determinant of where EV chargers might be located

[City of San Diego Zoning Data](https://data.sandiego.gov/datasets/zoning/)

In [5]:
zoning_data = gpd.read_file("shapefiles/zoning_datasd.shp")

zoning_data = zoning_data.to_crs(epsg=4326)  # Convert to WGS84 (EPSG:4326)

# Convert zone codes to types
zoning_categories = {
    'Commercial': ['CC', 'CN', 'CV', 'CP', 'CR', 'CCPD'],
    'Office': ['CO'],
    'Residential High': ['RH', 'RM-3', 'RM-4'],
    'Residential Medium': ['RM-2', 'RM-1'],
    'Residential Low': ['RS', 'RL'],
    'Residential Mixed': ['RMX'],
    'Industrial': ['IP', 'IL', 'IH', 'IS', 'IBT'],
    'Mixed Use': ['MU', 'EMX'],
    'Agricultural': ['AG', 'AR'],
    'Open Space': ['OS'],
    'Planned': ['BLPD', 'MBPD', 'GQPD', 'MPD', 'CUPD', 'LJPD', 'LJSPD'],
    'Transit': ['OTOP', 'OTRM', 'OTCC'],
    'Other': ['UNZONED'],
}

def map_zoning_category(zone_code):
    if isinstance(zone_code, str):  # Check if zone_code is a string
        for category, prefixes in zoning_categories.items():
            if any(zone_code.startswith(prefix) for prefix in prefixes):
                return category
    return 'Other'  # Return 'Other' for NaN or non-string values

zoning_data.fillna({'zone_name':'Unknown'}, inplace=True)
zoning_data['zone_type'] = zoning_data['zone_name'].apply(map_zoning_category)

In [6]:
# Proceed using ALL lots
lots = parking_lots_gdf.copy()

# intersections_gdf = intersections_gdf.set_crs("EPSG:4326", allow_override=True)
zoning_data = zoning_data.set_crs("EPSG:4326", allow_override=True)

# Perform the spatial join to map intersections to zoning data
lots = gpd.sjoin(lots, zoning_data, how="left", predicate="within")

# Drop irrelevant columns
lots = lots.drop(columns=['index_right','imp_date','ordnum','objectid'])

# Remove intersections with null zoning
lots.loc[pd.isna(lots['zone_type']), 'zone_type'] = "Multiple"

print(lots.shape)

(4269, 9)


### AFDC Charging Stations

Get an API key and follow the documentation at

[AFDC Charging Station API](https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/)

In [7]:
# Charger Data Query
url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json"
params = {
    'format': 'json',  # Output response format
    'api_key': 'Tbqhfv28h6gIIxbaMTByUNg4ByP2vf1E1A3XYgGa',  # Your developer API key
    'status': 'all',  # Return stations that match the given status
    # 'access': 'public',  # Return stations with the given access type
    'fuel_type': 'ELEC', # Return stations that supply any of the given fuel types
    'state': 'CA',
    'country': 'US',
}

response = requests.get(url, params=params)
if response.status_code == 200:

    data = response.json()

In [8]:
# CS - Charging Stations
cs = pd.DataFrame(data['fuel_stations'])
cs = cs[cs['zip'] != 'CA']
cs['zip'] = cs['zip'].astype(int)
cs = cs[cs['zip'].isin(sdge_zips)]
cs = cs[cs['access_code']=="public"]
cs = cs[[
    'id', 'access_code','facility_type','latitude','longitude','zip','ev_connector_types',
    'ev_dc_fast_num','ev_level1_evse_num','ev_level2_evse_num','ev_network'
]]
cs = cs.reset_index(drop=True)

In [9]:
# Create a geometry column for EV chargers
geometry = [Point(xy) for xy in zip(cs['longitude'], cs['latitude'])]
cs_gdf = gpd.GeoDataFrame(cs, geometry=geometry)
cs_gdf.set_crs(epsg=4326, inplace=True)

# Create a one-hot encoding of the 'ev_connector_types' column
# Step 1: Explode the 'ev_connector_types' column to create individual rows
cs_gdf_exploded = cs_gdf.explode('ev_connector_types', ignore_index=True)

# Step 2: One-hot encode the exploded 'ev_connector_types' column
one_hot_encoded = pd.get_dummies(cs_gdf_exploded['ev_connector_types'], prefix='connector')
connector_cols = list(one_hot_encoded.columns)

# Step 3: Combine the one-hot encoded DataFrame with the exploded DataFrame
cs_gdf_exploded = pd.concat([cs_gdf_exploded, one_hot_encoded], axis=1)
cs_gdf_exploded = cs_gdf_exploded.groupby('id')[connector_cols].sum().reset_index()

# Step 4: Aggregate back to the original DataFrame, summing up the one-hot columns
cs_gdf = cs_gdf.merge(cs_gdf_exploded, how='left', on='id')
cs_gdf = cs_gdf.drop(columns=['ev_connector_types'])

# Optional: Filter chargers by a specific zip code (e.g., 92122)
# zip_list = [92122, 92093, 92037, 92117, 92161, 92092, 92121, 92145]
# cs_gdf = cs_gdf[cs_gdf['zip'].isin(zip_list)]

# Check the resulting DataFrame
print(cs_gdf.shape)
cs_gdf.head()

(1698, 15)


,id,access_code,facility_type,latitude,longitude,zip,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,ev_network,geometry,connector_CHADEMO,connector_J1772,connector_J1772COMBO,connector_TESLA
0,6355,public,HOSPITAL,32.899470,-117.243000,92037,NaN,NaN,1.0,Non-Networked,POINT (-117.24300 32.89947),0,1,0,0
1,39886,public,CAR_DEALER,32.795896,-116.976156,92020,NaN,NaN,1.0,Non-Networked,POINT (-116.97616 32.79590),0,1,0,0
2,39889,public,CAR_DEALER,33.116492,-117.106837,92029,NaN,NaN,1.0,Non-Networked,POINT (-117.10684 33.11649),0,1,0,0
3,39915,public,CAR_DEALER,32.659001,-117.100836,91950,NaN,NaN,1.0,Non-Networked,POINT (-117.10084 32.65900),0,1,0,0
4,39916,public,CAR_DEALER,33.181953,-117.298334,92056,3.0,NaN,2.0,Non-Networked,POINT (-117.29833 33.18195),1,1,1,0


### Parking Lot - Charger  |  Proximity Calculations

- Assigns each charger to the nearest parking lot

For each parking lot:
- Calculate the number of chargers at the nearest station
- Calculate number of stations within a n meter radius (set to 200 meters)
- Calculate the distance to the nearest charger

In [10]:
%%time
# Map Chargers to Parking Lots

def nearest_lot(charger_row, lots_gdf):
    charger_coords = (charger_row['latitude'], charger_row['longitude'])
    distances = lots_gdf.geometry.apply(lambda x: geodesic(charger_coords, (x.centroid.y, x.centroid.x)).meters if isinstance(x, (Point, Polygon, MultiPolygon)) else float('inf'))
    nearest_index = distances.idxmin()
    return lots_gdf.loc[nearest_index]

def chargers_within_radius(lot, chargers_df, radius=400/2):  # radius in meters (1 mile = 1609.34 meters)
    lot_coords = (lot.geometry.centroid.y, lot.geometry.centroid.x)
    chargers_coords = list(zip(chargers_df['latitude'], chargers_df['longitude']))
    distances = [geodesic(lot_coords, charger).meters for charger in chargers_coords]
    return sum(d <= radius for d in distances)

def distance_to_nearest_charger(lot, chargers_df):
    lot_coords = (lot.geometry.centroid.y, lot.geometry.centroid.x)
    chargers_coords = list(zip(chargers_df['latitude'], chargers_df['longitude']))
    distances = [geodesic(lot_coords, charger).meters for charger in chargers_coords]
    return min(distances)

# Apply function to assign nearest lot to each charger
chargers_with_lots = cs_gdf.apply(nearest_lot, lots_gdf=lots, axis=1)

cs_by_lots = cs_gdf.merge(chargers_with_lots[['osmid']], how='left', left_index=True, right_index=True)

type_cols = ['ev_dc_fast_num','ev_level1_evse_num','ev_level2_evse_num']

cs_by_lots = cs_by_lots.groupby('osmid')[type_cols+connector_cols].sum().reset_index()
lots_cs = lots.merge(cs_by_lots, how='left', on='osmid')
lots_cs[type_cols] = lots_cs[type_cols].fillna(0)
lots_cs[connector_cols] = lots_cs[connector_cols].fillna(0)
lots_cs['cs_total'] = lots_cs[type_cols].sum(axis=1)
lots_cs[connector_cols] = lots_cs[connector_cols] > 0

# Apply function to calculate the number of chargers within a radius for each intersection
lots_cs['chargers_in_radius'] = lots_cs.apply(chargers_within_radius, chargers_df=cs_gdf, axis=1)

# Apply function to calculate the nearest charger distance for each intersection
lots_cs['distance_to_nearest_charger'] = lots_cs.apply(distance_to_nearest_charger, chargers_df=cs_gdf, axis=1)

lots_cs.to_file('all_lots_cs.geojson', driver='GeoJSON')

print(lots_cs.shape)
lots_cs.head()

In [11]:
# # Read file from the code generated above
# lots_cs = gpd.read_file("all_lots_cs.geojson")
# lots_cs.head(2)

,osmid,amenity,access,fee,parking,capacity,zone_name,zone_type,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,connector_CHADEMO,connector_J1772,connector_J1772COMBO,connector_TESLA,cs_total,chargers_in_radius,distance_to_nearest_charger,geometry
0,290588762,parking,yes,no,surface,None,OP-1-1,Other,0.0,0.0,0.0,False,False,False,False,0.0,0,895.327405,POINT (-117.19731 32.83273)
1,316725319,parking,None,None,multi-storey,None,CC-3-9,Commercial,0.0,0.0,0.0,False,False,False,False,0.0,2,169.564097,POINT (-117.15981 32.74716)


### Traffic Volume Data
[City of San Diego Traffic Volumes](https://data.sandiego.gov/datasets/traffic-volumes/)

In [12]:
# Import roadway and traffic volume data from San Diego Gov
rl = gpd.read_file("data/roads_datasd.geojson")
print(rl.shape)
rl.head(2)

tc = pd.read_csv("data/traffic_counts_datasd.csv")
# print(tc.shape)
tc['date_count'] = pd.to_datetime(tc['date_count'])
tc_sorted = tc.sort_values(by=['street_name', 'date_count'], ascending=[True, False])
tc_recent = tc_sorted.drop_duplicates(subset='street_name')
print(tc_recent.shape)
tc_recent.head(2)

In [13]:
from shapely.geometry import MultiLineString
rl = rl.to_crs(epsg=4326)

# Merge based on street names and ensure the result is a GeoDataFrame
tc_rel = tc_recent[['id','street_name','total_count']]
rl_rel = rl[['rd20full','geometry']]
merged = tc_rel.merge(rl_rel, left_on='street_name', right_on='rd20full', how='inner')

# Convert the merged DataFrame to a GeoDataFrame
road_traffic = gpd.GeoDataFrame(merged, geometry='geometry')

# Group by 'id', aggregate geometries into MultiLineString, and calculate the average 'total_count'
road_traffic_grouped = road_traffic.groupby('street_name').agg({
    'geometry': lambda x: MultiLineString(x.tolist()),
    'total_count': 'mean'
}).reset_index()

# Convert the grouped DataFrame to a GeoDataFrame and set the CRS
road_traffic_grouped = gpd.GeoDataFrame(road_traffic_grouped, geometry='geometry')
road_traffic_grouped.crs = 'EPSG:4326'

# Export to file
road_traffic_grouped.to_file('road_traffic.geojson', driver='GeoJSON')

# Print the final road_traffic_grouped GeoDataFrame and count of NaN in 'total_count'
print(road_traffic_grouped['total_count'].isna().sum())

road_traffic_grouped.head(2)

In [14]:
# # Read file from the code generated above
# road_traffic_grouped = gpd.read_file('road_traffic.geojson')

In [15]:
%%time
road_traffic_grouped = road_traffic_grouped.to_crs('EPSG:4326')
lots_cs = lots_cs.to_crs('EPSG:4326')

cs_lots_traffic = lots_cs.copy()

# Re-project to a projected CRS for accurate buffer operation
buffered_cs_lots = lots_cs.to_crs('EPSG:3857')
road_traffic_projected = road_traffic_grouped.to_crs('EPSG:3857')

# Buffer the parking lots to consider "adjacent" roads within a given distance (e.g., 5 meters)
buffered_cs_lots['geometry'] = buffered_cs_lots['geometry'].buffer(50)

# Calculate the total adjacent traffic volume for each parking lot
def calculate_total_adjacent_traffic(parking_lot, road_traffic_grouped):
    total_traffic = 0
    for road in road_traffic_grouped.itertuples():
        if parking_lot.intersects(road.geometry):
            total_traffic += road.total_count
    return total_traffic

# Add a new column to cs_lots for total adjacent traffic
cs_lots_traffic['traffic'] = buffered_cs_lots.apply(
    lambda row: calculate_total_adjacent_traffic(row['geometry'], road_traffic_projected), axis=1
).astype(int)  # Calculate total adjacent traffic using roads.

# Re-project back to the original CRS
cs_lots_traffic = cs_lots_traffic.to_crs('EPSG:4326')

# Print the results
cs_lots_traffic.head(2)

CPU times: user 39.5 s, sys: 56.3 ms, total: 39.5 s
Wall time: 39.5 s


,osmid,amenity,access,fee,parking,capacity,zone_name,zone_type,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,connector_CHADEMO,connector_J1772,connector_J1772COMBO,connector_TESLA,cs_total,chargers_in_radius,distance_to_nearest_charger,geometry,traffic
0,290588762,parking,yes,no,surface,None,OP-1-1,Other,0.0,0.0,0.0,False,False,False,False,0.0,0,895.327405,POINT (-117.19731 32.83273),0
1,316725319,parking,None,None,multi-storey,None,CC-3-9,Commercial,0.0,0.0,0.0,False,False,False,False,0.0,2,169.564097,POINT (-117.15981 32.74716),4810


### ZIP Code Tabulation Areas

Omitted from this code is an additional line of code which reads in the full ZCTA File from the [Census.gov Tigerline Directory](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html) and filters the ZCTA by zip codes in the SDG&E territory. This code is ommitted because the operation was performed locally to work with the 500MB ZCTA file. The resulting file below is the ZCTA with the zip code itself filtered by 'sdge_zips'.

In [16]:
zcta_shp = "shapefiles/sdge_zcta.shp"
zcta = gpd.read_file(zcta_shp)

In [34]:
# Map each parking lot to a ZIP-code
cs_lots_traffic_zcta = cs_lots_traffic.to_crs(epsg=4326)
zcta = zcta.to_crs(epsg=4326)

# GPD Spatial Join
cs_lots_traffic_zcta = gpd.sjoin(cs_lots_traffic_zcta, zcta[['geometry', 'ZCTA5CE20']], how='left', predicate='intersects')

cs_lots_traffic_zcta = cs_lots_traffic_zcta.rename(columns={'ZCTA5CE20': 'zcta'})
cs_lots_traffic_zcta = cs_lots_traffic_zcta.drop(columns=['index_right'])

cs_lots_traffic_zcta.head(2)

,osmid,amenity,access,fee,parking,capacity,zone_name,zone_type,ev_dc_fast_num,ev_level1_evse_num,...,connector_CHADEMO,connector_J1772,connector_J1772COMBO,connector_TESLA,cs_total,chargers_in_radius,distance_to_nearest_charger,geometry,traffic,zcta
0,290588762,parking,yes,no,surface,None,OP-1-1,Other,0.0,0.0,...,False,False,False,False,0.0,0,895.327405,POINT (-117.19731 32.83273),0,92117
1,316725319,parking,None,None,multi-storey,None,CC-3-9,Commercial,0.0,0.0,...,False,False,False,False,0.0,2,169.564097,POINT (-117.15981 32.74716),4810,92103


### Shopping Clusters

In [18]:
shops_gdf = gpd.read_file("shops.geojson", driver="GeoJSON")
print(shops_gdf.shape)
shops_gdf.head(2)

(3764, 170)


,id,geo_type,latitude,longitude,name,postal_code,nodes,nearest_charger_distance,charger_density,shop_Guitar_manufacturing_and_sales,...,shop_water,shop_weapons,shop_weight_loss,shop_wholesale,shop_window_blind,shop_wine,shop_yes,cluster,in_top_100,geometry
0,273593057,node,32.866410,-117.232087,Ralphs,92037,None,0.105186,14,False,...,False,False,False,False,False,False,False,41,True,POINT (-117.23209 32.86641)
1,273593515,node,32.865636,-117.232406,Trader Joe's,92037,None,0.162983,11,False,...,False,False,False,False,False,False,False,41,True,POINT (-117.23241 32.86564)


In [31]:
# Filter out rows where 'cluster' is NaN
shops_clustered = shops_gdf.dropna(subset=['cluster'])
shops_clustered = gpd.GeoDataFrame(shops_clustered, geometry='geometry')

# Group by 'cluster' and calculate count and centroid
shops_cluster_gdf = shops_clustered.dissolve(by='cluster', as_index=False)

# Re-projection
shops_projected = shops_cluster_gdf.to_crs("EPSG:26911")
shops_projected['geometry'] = shops_projected.centroid

# Project to original crs
shops_cluster_gdf = shops_projected.to_crs(shops_gdf.crs)

# Add store count per cluster
shops_cluster_gdf['store_count'] = shops_clustered.groupby('cluster').size()

# # Reset index to ensure 'cluster' remains a column
shops_cluster_gdf.head(2)

,cluster,geometry,id,geo_type,latitude,longitude,name,postal_code,nodes,nearest_charger_distance,...,shop_watches,shop_water,shop_weapons,shop_weight_loss,shop_wholesale,shop_window_blind,shop_wine,shop_yes,in_top_100,store_count
0,0,POINT (-117.12900 32.83241),2769614157,node,32.830819,-117.138491,Foot Massage,92123,None,0.459762,...,False,False,False,False,False,False,False,False,True,48
1,1,POINT (-117.70236 33.57437),4400587386,node,33.577876,-117.702288,Nothing Bundt Cakes,92653,None,1.199252,...,False,False,False,False,False,False,False,False,True,27


In [39]:
from shapely.ops import nearest_points
from scipy.spatial import cKDTree
import numpy as np

# Ensure both GeoDataFrames use a projected CRS (e.g., UTM Zone 11N)
projected_crs = "EPSG:26911"

cs_lots_shops = cs_lots_traffic_zcta.to_crs(projected_crs)
shops_cluster_gdf = shops_cluster_gdf.to_crs(projected_crs)

# Convert parking lots (if polygon/multipolygon) to centroids
cs_lots_shops['lot_centroid'] = cs_lots_shops.geometry.centroid

# Extract array of shopping cluster points for spatial lookup
shop_cluster_coords = np.array([(geom.x, geom.y) for geom in shops_cluster_gdf.geometry])
shop_cluster_tree = cKDTree(shop_cluster_coords)

# Function to find the nearest cluster
def find_nearest_cluster(lot_geom):
    lot_x, lot_y = lot_geom.x, lot_geom.y  # Extract centroid coordinates
    dist, idx = shop_cluster_tree.query([lot_x, lot_y])  # Query nearest
    nearest_cluster = shops_cluster_gdf.iloc[idx]['cluster']
    return pd.Series([nearest_cluster, dist])

# Apply nearest neighbor search to each parking lot
cs_lots_shops[['nearest_cluster', 'proximity_meters']] = cs_lots_shops['lot_centroid'].apply(find_nearest_cluster)

# Drop temporary centroid column
cs_lots_shops = cs_lots_shops.drop(columns=['lot_centroid'])

# Convert back to original CRS for consistency
cs_lots_shops = cs_lots_shops.to_crs(cs_lots_traffic_zcta.crs)

# Upload gdf to file
cs_lots_shops.to_file('cs_lots_shops.geojson', driver='GeoJSON')

cs_lots_shops.head(2)

,osmid,amenity,access,fee,parking,capacity,zone_name,zone_type,ev_dc_fast_num,ev_level1_evse_num,...,connector_J1772COMBO,connector_TESLA,cs_total,chargers_in_radius,distance_to_nearest_charger,geometry,traffic,zcta,nearest_cluster,proximity_meters
0,290588762,parking,yes,no,surface,None,OP-1-1,Other,0.0,0.0,...,False,False,0.0,0,895.327405,POINT (-117.19731 32.83273),0,92117,75.0,737.953030
1,316725319,parking,None,None,multi-storey,None,CC-3-9,Commercial,0.0,0.0,...,False,False,0.0,2,169.564097,POINT (-117.15981 32.74716),4810,92103,225.0,209.424786


## Implemented in Pop/Inc Branches

### Demographic Data

In [36]:
pop_gdf = gpd.read_file("population.geojson", driver="GeoJSON")
income_gdf = gpd.read_file("income.geojson", driver="GeoJSON")
pop_gdf.head(2)

,GEOID,Population,NAME,state,county,tract,geometry
0,06073010012,5199.0,"Census Tract 100.12, San Diego County, California",06,073,010012,"POLYGON ((-13031330.99000 3837056.99000, -1303..."
1,06073010009,6978.0,"Census Tract 100.09, San Diego County, California",06,073,010009,"POLYGON ((-13032295.57000 3834831.28000, -1303..."


## Future Implementations

### DMV Registration Data

Gather information on the local registration information of vehicles according to the DMV

[Vehicle Fuel Type Count by Zip Code](https://data.ca.gov/dataset/vehicle-fuel-type-count-by-zip-code)

In [38]:
dmv_data = pd.read_csv("vehicle-fuel-type-count-by-zip-code-20231.csv")
dmv_data.head(2)

,Date,ZIP Code,Model Year,Fuel,Make,Duty,Vehicles
0,12/31/2023,91302,<2010,Unk,OTHER/UNK,Unk,490
1,12/31/2023,93631,<2010,Unk,OTHER/UNK,Unk,414
